# Readme

Pre-requisites
*   Set up weight and bias account: [Wandb](https://wandb.ai/site/)
*   Upload dataset on drive following the hierarchy below: <br>
<pre>
MyDrive
|_ Final_project
   |_ 2000dataset.zip
</pre>

*   Upload the zip containing the code for **fine tuning and inference** in drive following the herachy below:
<pre>
MyDrive
|_ Final_project
   |_ bsrgan
      |_ BSRGAN.zip
      |_ BSRGAN_Inference.zip
</pre>

*   for **inference** using generator obtained after fine tuning on the dataset, ensure 5000_E is at the hierarchy below:

 <pre>
MyDrive
|_ Final_project
   |_ bsrgan
      |_ 5000_E.pth
</pre>


**Fine-Tuning**: Execute steps 1 and 2 in order. <br> <br>
**Inference**:
*   The bsrgan inference code and the encoder is required for inference. Unlike other models which requires Generator, BSRGAN is more stable with Encoder..
*   Run step 1 and Step 3. *Note: At step 3.3, make sure model_path and test_image_path have the correct paths.*


# 1 Install dependencies and upload BSRGAN code

In [ ]:
!pip install wandb -qqq
!pip install basicsr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 27.5 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=1f4306cca1960bc0f169121d5e512e9663f8a96f2d13aa7552629d9e633ec9c4
  Stored in directory: /root/.cache/pip/wheels/9a/e3/e4/58f29bfabb622dd40b6d9839318ce5bf092062b81ca3aa19ea
Successfully built basicsr


In [ ]:
#basicssr contains a file degradation.py where the import library is obsolete
#and has to be replaced
#https://github.com/xinntao/Real-ESRGAN/issues/801

file_path = "/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py"
line_to_replace = "from torchvision.transforms.functional_tensor import rgb_to_grayscale"
new_line = "from torchvision.transforms.functional import rgb_to_grayscale"

with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [new_line if line.strip() == line_to_replace else line for line in lines]

with open(file_path, 'w') as file:
    file.writelines(lines)

print(f"Replacing line '{line_to_replace}' with '{new_line}' in file {file_path}")

Replacing line 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' with 'from torchvision.transforms.functional import rgb_to_grayscale' in file /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py


In [ ]:
from google.colab import files, drive
import zipfile
import os
import json
import shutil
import basicsr
import wandb

In [ ]:
#mount google drive to colab
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# log in to Weights & Biases ; skip for if doing inference only
wandb.login()

# 2 BSRGAN Fine-tuning

## 2.1 Get code and requirements

In [ ]:
code_zip_path = '/content/drive/MyDrive/Final_project/bsrgan/BSRGAN.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path

total 12
drwx------ 5 root root 4096 Sep 18 07:26 drive
drwxrwxrwx 7 root root 4096 Aug 19 11:06 KAIR
drwxr-xr-x 1 root root 4096 Sep 16 13:40 sample_data


In [ ]:
!pip install -r /content/KAIR/requirement.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 12.0 MB/s eta 0:00:00


## 2.2 Download pre-trained models

In [ ]:
%cd KAIR
!python main_download_pretrained_models.py --models "BSRGAN" --model_dir "model_zoo"

/content/KAIR
trying to download ['BSRGAN']
downloading [model_zoo/BSRGAN.pth] ...
done!
downloading [model_zoo/BSRNet.pth] ...
done!
downloading [model_zoo/BSRGANx2.pth] ...
done!


In [ ]:
#This step is required as network will fine-tune from pre-trained models only if they are in the the models folder from superresolution
#They also need to be renamed to 1_G as the model understands it like a checkpoint saved from previous exec.

# Source dir
base_dir = "/content/KAIR"

# Source folder
models_src = os.path.join(base_dir, "model_zoo")

# Destination folders
gan_dst = os.path.join(base_dir, "superresolution/bsrgan_x4_gan/models")

# Create destination folders
os.makedirs(gan_dst, exist_ok=True)

# Move to GAn folder
model_map = {
    "BSRGAN.pth": gan_dst
}

for model_name, dst in model_map.items():
    src_path = os.path.join(models_src, model_name)
    dst_path = os.path.join(dst, "1_G.pth")  # rename

    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)
        print(f"Moved {model_name} to {dst_path}")
    else:
        print(f"{model_name} not found in {models_src}")

print("Models placed and renamed correctly.")


Moved BSRGAN.pth to /content/KAIR/superresolution/bsrgan_x4_gan/models/1_G.pth
Models placed and renamed correctly.


## 2.3 Upload Dataset from Drive

In [ ]:
dataset_zip_path = '/content/drive/MyDrive/Final_project/2000dataset.zip'
!unzip -q $dataset_zip_path -d $unzip_path

## 2.4 BSRGAN configuration and fine-tuning

In [ ]:
# Define config content as a string
train_bsrgan_x4_gan = """
{
  "task": "bsrgan_x4_gan",
  "model": "gan",
  "gpu_ids": [0],
  "scale": 4,
  "n_channels": 3,

  "path": {
    "root": "superresolution",
    "pretrained_netG": "/content/KAIR/superresolution/bsrgan_x4_gan/models/1_G.pth",
    "pretrained_netD": null,
    "pretrained_netE": null
  },

  "datasets": {
    "train": {
      "name": "train_dataset",
      "dataset_type": "sr",
      "dataroot_H": "/content/dataset/train/hr",
      "dataroot_L": "/content/dataset/train/lr",
      "H_size": 256,
      "shuffle_prob": 0.1,
      "lq_patchsize": 64,
      "use_sharp": false,
      "dataloader_shuffle": true,
      "dataloader_num_workers": 2,
      "dataloader_batch_size": 2
    },
    "test": {
      "name": "test_dataset",
      "dataset_type": "sr",
      "H_size": 256,
      "lq_patchsize": 64,
      "use_sharp": false,
      "dataroot_H": "/content/dataset/val/hr",
      "dataroot_L": "/content/dataset/val/lr"
    }
  },

  "netG": {
    "net_type": "rrdbnet",
    "in_nc": 3,
    "out_nc": 3,
    "nf": 64,
    "nb": 23,
    "gc": 32,
    "ng": 2,
    "reduction": 16,
    "act_mode": "L",
    "bias": true,
    "init_type": "orthogonal",
    "init_bn_type": "uniform",
    "init_gain": 0.2
  },

  "netD": {
    "net_type": "discriminator_unet",
    "in_nc": 3,
    "base_nc": 64,
    "n_layers": 3,
    "norm_type": "spectral",
    "init_type": "orthogonal",
    "init_bn_type": "uniform",
    "init_gain": 0.2
  },

  "train": {
    "G_lossfn_type": "l1",
    "G_lossfn_weight": 1,
    "F_lossfn_type": "l1",
    "F_lossfn_weight": 0.1,
    "F_feature_layer": [2,7,16,25,34],
    "F_weights": [0.1,0.1,1.0,1.0,1.0],
    "F_use_input_norm": true,
    "F_use_range_norm": false,
    "gan_type": "lsgan",
    "D_lossfn_weight": 0.2,
    "E_decay": 0.999,
    "D_init_iters": 100,
    "G_optimizer_type": "adam",
    "G_optimizer_lr": 5e-6,
    "G_optimizer_wd": 0,
    "D_optimizer_type": "adam",
    "D_optimizer_lr": 5e-6,
    "D_optimizer_wd": 0,
    "G_scheduler_type": "MultiStepLR",
    "G_scheduler_milestones": [2500],
    "G_scheduler_gamma": 0.5,
    "G_optimizer_reuse": true,
    "D_scheduler_type": "MultiStepLR",
    "D_scheduler_milestones": [2500],
    "D_scheduler_gamma": 0.8,
    "D_optimizer_reuse": false,
    "G_param_strict": true,
    "D_param_strict": true,
    "E_param_strict": true,
    "manual_seed": 8747,
    "checkpoint_test": 100,
    "checkpoint_save": 500,
    "checkpoint_print": 100
  }
}
"""

# Overwrite the file with new content
config_path = "/content/KAIR/options/train_bsrgan_x4_gan.json"
with open(config_path, "w") as f:
    f.write(train_bsrgan_x4_gan)

print(f"Config overwritten at {config_path}")


Config overwritten at /content/KAIR/options/train_bsrgan_x4_gan.json


In [ ]:
%cd /content/KAIR/
!python main_train_gan.py --opt options/train_bsrgan_x4_gan.json --wandb_project BSRGAN --wandb_name BSRGAN_finetune

/content/KAIR
export CUDA_VISIBLE_DEVICES=0
number of GPUs is: 1
wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.
wandb: Currently logged in as: dsha43925 (dsha43925-middlesex-university-mauritius) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: ⣽ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.21.3
wandb: Run data is saved locally in /content/KAIR/wandb/run-20250918_103023-ms2zcay0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run BSRGAN_finetune
wandb: ⭐️ View project at https://wandb.ai/dsha43925-middlesex-university-mauritius/BSRGAN
wandb: 🚀 View run at https://wandb.ai/dsha43925-middlesex-university-mauritius/BSRGAN/runs/ms2zcay0
LogHandlers setup!
25-09-18 10:30:25.128 :   task: bsrgan_x4_gan
  model: gan
  gpu_ids: [0]
  scale: 4
  n_ch

# 3 Inference with BSRGAN

## 3.1 Upload Inference code

In [ ]:
code_zip_path = '/content/drive/MyDrive/Final_project/bsrgan/BSRGAN_Inference.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path

## 3.2 Upload test dataset

In [ ]:
#Upload the zip file containing low resolution images for inference.
#code_zip_path = '/content/drive/MyDrive/Final_project/testImages.zip'
code_zip_path = '/content/drive/MyDrive/Final_project/LRInferenceImages.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path

total 20
drwxrwxrwx 5 root root 4096 Aug  8 18:30 BSRGAN_Inference
drwx------ 5 root root 4096 Sep 14 22:35 drive
drwxrwxrwx 7 root root 4096 Aug 19 11:06 KAIR
drwxr-xr-x 1 root root 4096 Sep  9 13:46 sample_data
drwxrwxrwx 7 root root 4096 Sep 11 19:39 testImages


## 3.3 Inference

In [ ]:
#simple inference on LRInferenceImages
image_path = "/content/LRInferenceImages"
model_path = "/content/drive/MyDrive/Final_project/bsrgan/5000_E.pth"
!python /content/BSRGAN_Inference/main_test_bsrgan.py --model_path "{model_path}" --test_image_path "{image_path}"

LogHandlers setup!
25-09-23 19:24:03.111 :       Model Name : BSRGAN
25-09-23 19:24:03.127 :           GPU ID : 0
[3, 3, 64, 23, 32, 4]
25-09-23 19:24:10.775 :       Input Path : /content/LRInferenceImages/
25-09-23 19:24:10.776 :      Output Path : /content/LRInferenceImages/_results_x4
25-09-23 19:24:10.776 : ---1 --> BSRGAN --> x4--> AirbusDhirubhai_AmbaniSolarPlant.png
25-09-23 19:24:12.252 : ---2 --> BSRGAN --> x4--> AirbusRotterdamPort.png
25-09-23 19:24:14.597 : ---3 --> BSRGAN --> x4--> AirbusSantiagoAirport.png
25-09-23 19:24:15.932 : ---4 --> BSRGAN --> x4--> MaxarIndonesia.png
25-09-23 19:24:17.829 : ---5 --> BSRGAN --> x4--> MaxarYellowStone.png
25-09-23 19:24:21.043 : ---6 --> BSRGAN --> x4--> TurkeyEarthquake.png
25-09-23 19:24:22.259 : ---7 --> BSRGAN --> x4--> italystadiumAirbus.png
25-09-23 19:24:23.955 : ---8 --> BSRGAN --> x4--> pittsburg.png
25-09-23 19:24:25.181 : ---9 --> BSRGAN --> x4--> plantation.png


In [ ]:
import shutil
import os
from google.colab import files

# For all files in the results directory
results_dir = "/content/LRInferenceImages/_results_x4"
zip_path = "/content/bsrganinference5000E.zip"

# Create zip archive
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', results_dir)

# Download the zip file
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#inference on the test dataset
test_image_path = "/content/testImages"
folders = os.listdir(test_image_path)
model_path = "/content/drive/MyDrive/Final_project/bsrgan/5000_E.pth"
#model_path = "/content/KAIR/model_zoo/BSRGAN.pth"
for folder in folders:
    image_path = os.path.join(test_image_path, folder)
    if os.path.isdir(image_path):
        print(f"Processing {folder}")
        !python /content/BSRGAN_Inference/main_test_bsrgan.py --model_path "{model_path}" --test_image_path "{image_path}"

Processing buildings
LogHandlers setup!
25-09-14 22:36:24.974 :       Model Name : BSRGAN
25-09-14 22:36:24.996 :           GPU ID : 0
[3, 3, 64, 23, 32, 4]
25-09-14 22:36:26.104 :       Input Path : /content/testImages/buildings/
25-09-14 22:36:26.104 :      Output Path : /content/testImages/buildings/_results_x4
25-09-14 22:36:26.105 : ---1 --> BSRGAN --> x4--> buildings2_patch_0.png
25-09-14 22:36:27.045 : ---2 --> BSRGAN --> x4--> buildings2_patch_1.png
25-09-14 22:36:27.112 : ---3 --> BSRGAN --> x4--> buildings2_patch_10.png
25-09-14 22:36:27.162 : ---4 --> BSRGAN --> x4--> buildings2_patch_11.png
25-09-14 22:36:27.208 : ---5 --> BSRGAN --> x4--> buildings2_patch_12.png
25-09-14 22:36:27.253 : ---6 --> BSRGAN --> x4--> buildings2_patch_13.png
25-09-14 22:36:27.298 : ---7 --> BSRGAN --> x4--> buildings2_patch_14.png
25-09-14 22:36:27.342 : ---8 --> BSRGAN --> x4--> buildings2_patch_15.png
25-09-14 22:36:27.380 : ---9 --> BSRGAN --> x4--> buildings2_patch_16.png
25-09-14 22:36:27.41

In [ ]:
base_folder = "/content/testImages"
categories = ["buildings", "desert", "snowregion", "vegetation", "water"]

# Prepare a destination folder for organised results
organised_folder = "/content/organised_results"
os.makedirs(organised_folder, exist_ok=True)

for cat in categories:
    src_folder = os.path.join(base_folder, cat, "_results_x4")
    dst_folder = os.path.join(organised_folder, cat)
    os.makedirs(dst_folder, exist_ok=True)

    if os.path.exists(src_folder):
        for fname in os.listdir(src_folder):
            fpath = os.path.join(src_folder, fname)
            if os.path.isfile(fpath):
                shutil.copy(fpath, os.path.join(dst_folder, fname))

# Zip all categories together
zip_path = "/content/bsrganNotFinetuned"
shutil.make_archive(zip_path, 'zip', organised_folder)

# Download the zip
files.download(f"{zip_path}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>